# 1.0 Imports

In [93]:
import os
import cv2 as cv
import numpy  as np
import ntpath
import math
import pickle
from scipy.special import softmax


from sklearn.svm import SVC

from IPython.core.display import HTML

## 1.1 Pre settings

In [3]:
display( HTML( '<style>.container { width:100% !important; }</style>') )


In [156]:
class ReconhecimentoFacial :
     
#     model_dnn_type = "OPENFACE"
#     svc = None
#     image_labels = None 
    
    
    def __init__(self, model_name):
        
        
        self.model_dnn_type = model_name
        self.detecta_rosto = cv.dnn.readNetFromCaffe('../modelosDNN/deploy.prototxt', '../modelosDNN/res10_300x300_ssd_iter_140000.caffemodel')
        self.detecta_rosto.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
        self.detecta_rosto.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)


        #extracao keypoint rostos OPENFACE
        self.open_face = cv.dnn.readNetFromTorch("../modelosDNN/openface/openface.nn4.small2.v1.t7");
        self.open_face.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
        self.open_face.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)


        self.face_net = cv.dnn.readNetFromTensorflow("../modelosDNN/facenet/facenet_graph_final.pb");
        self.face_net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
        self.face_net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)
        
        model_name = "../modeloSVM/SVC_model.pkl"
        if os.path.exists(model_name):
            with open(model_name, 'rb') as file:  
                self.svc = pickle.load(file)
        
        path_model = "../modeloSVM/etiquetas.pkl"
        if os.path.exists(path_model):
            with open(path_model, 'rb') as file:  
                self.image_labels = pickle.load(file)

     
    def absoluteFilePaths(directory):
        for dirpath,_,filenames in os.walk(directory):
            for f in filenames:
                yield os.path.abspath(os.path.join(dirpath, f))
                
    def ExtractDatasetData(self, path = "../imagens"):
        image_labels={}
        files=[]
        indice_etiqueta_lista = []
        for file in absoluteFilePaths(path):
            #print(ntpath.basename(file))
            files.append(file)
            nome_arquivo = ntpath.basename(file)
            index_1 = nome_arquivo.find("@")
            index_2 = nome_arquivo.find("-")

            indice_etiqueta = int(nome_arquivo[:index_1])
            indice_etiqueta_lista.append(indice_etiqueta)
            label_etiqueta = nome_arquivo[index_1+1:index_2]

            if indice_etiqueta not in image_labels :
                image_labels[indice_etiqueta] = label_etiqueta

        return image_labels,files,indice_etiqueta_lista
    
   
    def FindFaceinImage(self, path_image="", img = None):
    #correr o dataset e aplicar o modelo para recuperar as keypoints


        if img is None:
            img = cv.imread(path_image)
            if img is None:
                return None


        blob = cv.dnn.blobFromImage(img, 1.0,(300,300),(104.0, 177.0, 123.0), False, False)
        self.detecta_rosto.setInput(blob)
        detections = self.detecta_rosto.forward()

        return img, detections

        
    
    def ExtractFaceRectangle(self, img, detections, confidence_threshold=0.5):
        # loop over the detections
        boxes = []
        (h, w) = img.shape[:2]
        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with the
            # prediction
            confidence = detections[0, 0, i, 2]
            # filter out weak detections by ensuring the `confidence` is
            # greater than the minimum confidence
            if confidence > confidence_threshold:
                # compute the (x, y)-coordinates of the bounding box for the
                # object
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                startX = max(0, min(startX, w - 1));
                startY = max(0, min(startY, h - 1));
                endX = max(0, min(endX, w - 1));
                endY = max(0, min(endY, h - 1));

                boxes.append([(startX, startY, endX, endY),confidence])

        return boxes
    
    def DrawImageFaceRectangle(self, img, box, confidence, text=None):
        # draw the bounding box of the face along with the associated
                # probability
        img2 = img.copy()
        if text is None:
            text = "{:.2f}%".format(confidence * 100)
        else :
            text = text + "  {:.2f}%".format(confidence * 100)
        #(startX, startY, endX, endY) = box[0],box[1],box[2],box[3]
        (startX, startY, endX, endY) = box
        y = endY +20 
        cv.rectangle(img2, (startX, startY), (endX, endY),(0, 255, 0), 1)
        cv.putText(img2, text, (startX-20, y),cv.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 0), 2)

        return img2;


    def TrainSVC(self, x_train, y_train):
        self.svc = SVC(kernel = "rbf", gamma=0.1, class_weight="balanced", decision_function_shape = "ovr")
        self.svc.fit(x_train,y_train)

        model_name = "../modeloSVM/SVC_model.pkl"  

        with open(model_name, 'wb') as file:  
            pickle.dump(self.svc, file)
    
    def ExtractFacePoints(self, img):
        vec = None
        if model_dnn_type =="OPENFACE":

            blob_f = cv.dnn.blobFromImage(img, 1.0/255.0,(96,96),(0, 0, 0), True, False)
            self.open_face.setInput(blob_f)
            vec = self.open_face.forward()
            vec = cv.normalize(vec, None,-1, 1, cv.NORM_MINMAX)
        elif model_dnn_type =="FACENET":

            blob_f = cv.dnn.blobFromImage(img, 1.0/255.0,(160,160),(0, 0, 0), True, False)
            self.face_net.setInput(blob_f)
            vec = self.face_net.forward()
            vec = cv.normalize(vec, None,-1, 1, cv.NORM_MINMAX)
        return vec
    
    
    def TrainModelfull(self):

        self.image_labels,files,indice_etiqueta_lista = ExtractDatasetData()

        indexes_ok=[]
        array_keypoints=[]
        for i,f in enumerate(files):

            img, detections = FindFaceinImage(path_image=f)
            if detections is not None:
                indexes_ok.append(i)
                boxes = ExtractFaceRectangle(img,detections,0.5)
                for b in boxes:
                    #DrawImageFaceRectangle(img,b[0],b[1],)
                    (startX, startY, endX, endY) = b[0]
                    points = ExtractFacePoints(img[startY:endY,startX:endX])
                    if points is not None:
                        array_keypoints.append(points)


        [indice_etiqueta_lista[i] for i in indexes_ok]

        x_train = np.array([arr.flatten() for arr in array_keypoints])
        y_train = np.array(indice_etiqueta_lista)
        y_train = y_train.reshape(y_train.shape[0],1).ravel()

        TrainSVC(x_train,y_train)
        model_name = "../modeloSVM/etiquetas.pkl"
        with open(model_name, 'wb') as file:  
            pickle.dump(self.image_labels, file)

        print("Train finished")
        
        
    def PredictFace(self, img):
        img, detections = FindFaceinImage(img=img)
        if detections is not None:
            boxes = ExtractFaceRectangle(img,detections,0.5)
            for b in boxes:
                (startX, startY, endX, endY) = b[0]
                points = ExtractFacePoints(img[startY:endY,startX:endX])
                if points is not None:
                    id_ = self.svc.predict(points)
                    df = self.svc.decision_function(points)
                    result = softmax(df)
                    etiqueta = image_labels[id_[0]]


                    return DrawImageFaceRectangle(img,b[0],np.max(result),etiqueta)


## 1.2 Load Models DNN

In [4]:
#modelo detecta rostos nas imagens
detecta_rosto = cv.dnn.readNetFromCaffe('../modelosDNN/deploy.prototxt', '../modelosDNN/res10_300x300_ssd_iter_140000.caffemodel')
detecta_rosto.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
detecta_rosto.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)


#extracao keypoint rostos OPENFACE
open_face = cv.dnn.readNetFromTorch("../modelosDNN/openface/openface.nn4.small2.v1.t7");
open_face.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
open_face.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)


face_net = cv.dnn.readNetFromTensorflow("../modelosDNN/facenet/facenet_graph_final.pb");
face_net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
face_net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [145]:
r = ReconhecimentoFacial("OPENFACE")

In [147]:
r.TrainModelfull()

Train finished


# 2.0 General class funcions

In [108]:
model_dnn_type = "OPENFACE"
svc = None
image_labels = None

In [111]:
model_name = "../modeloSVM/SVC_model.pkl"
if os.path.exists(model_name):
    with open(model_name, 'rb') as file:  
        svc = pickle.load(file)
        
path_model = "../modeloSVM/etiquetas.pkl"
if os.path.exists(path_model):
    with open(path_model, 'rb') as file:  
        image_labels = pickle.load(file)


In [79]:
def absoluteFilePaths(directory):
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))

In [80]:
def ExtractDatasetData(path = "../imagens"):
    image_labels={}
    files=[]
    indice_etiqueta_lista = []
    for file in absoluteFilePaths(path):
        #print(ntpath.basename(file))
        files.append(file)
        nome_arquivo = ntpath.basename(file)
        index_1 = nome_arquivo.find("@")
        index_2 = nome_arquivo.find("-")

        indice_etiqueta = int(nome_arquivo[:index_1])
        indice_etiqueta_lista.append(indice_etiqueta)
        label_etiqueta = nome_arquivo[index_1+1:index_2]
    #     print(nome_arquivo)
    #     print(indice_etiqueta)
    #     print(label_etiqueta)
    #     print("\n")

        if indice_etiqueta not in image_labels :
            image_labels[indice_etiqueta] = label_etiqueta

    return image_labels,files,indice_etiqueta_lista

In [81]:
#correr o dataset e aplicar o modelo para recuperar as keypoints
def FindFaceinImage(path_image="", img = None):

#file = "C:\\KLEBER\\PROJETOS PYTHON\\ReconhecimentoFacialTelegram\\imagens\\1@BOLSONARO-1.jpg"
    if img is None:
        img = cv.imread(path_image)
        if img is None:
            return None
    
    #cv.imshow("OpenCV Image Reading", img)
    #cv.waitKey(0)
    #cv.destroyAllWindows()
    blob = cv.dnn.blobFromImage(img, 1.0,(300,300),(104.0, 177.0, 123.0), False, False)
    detecta_rosto.setInput(blob)
    detections = detecta_rosto.forward()
    #cv.imshow("OpenCV Image Reading", img)
    #cv.waitKey(0)
    return img, detections
#cv.destroyAllWindows()


In [82]:
def ExtractFaceRectangle(img, detections, confidence_threshold=0.5):
    # loop over the detections
    boxes = []
    (h, w) = img.shape[:2]
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > confidence_threshold:
            # compute the (x, y)-coordinates of the bounding box for the
            # object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            startX = max(0, min(startX, w - 1));
            startY = max(0, min(startY, h - 1));
            endX = max(0, min(endX, w - 1));
            endY = max(0, min(endY, h - 1));
         
            boxes.append([(startX, startY, endX, endY),confidence])

    return boxes      

In [130]:
def DrawImageFaceRectangle(img, box, confidence, text=None):
    # draw the bounding box of the face along with the associated
            # probability
    img2 = img.copy()
    if text is None:
        text = "{:.2f}%".format(confidence * 100)
    else :
        text = text + "  {:.2f}%".format(confidence * 100)
    #(startX, startY, endX, endY) = box[0],box[1],box[2],box[3]
    (startX, startY, endX, endY) = box
    y = endY +20 
    cv.rectangle(img2, (startX, startY), (endX, endY),(0, 255, 0), 1)
    cv.putText(img2, text, (startX-20, y),cv.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 0), 2)
    # show the output image
#     cv.imshow("Output", img2)
#     cv.waitKey(0)
#     cv.destroyAllWindows()
    return img2;

In [84]:
def TrainSVC(x_train, y_train):
    svc = SVC(kernel = "rbf", gamma=0.1, class_weight="balanced", decision_function_shape = "ovr")
    svc.fit(x_train,y_train)
    
    model_name = "../modeloSVM/SVC_model.pkl"  

    with open(model_name, 'wb') as file:  
        pickle.dump(svc, file)
    
    

In [85]:
def ExtractFacePoints(img):
    vec = None
    if model_dnn_type =="OPENFACE":
    
        blob_f = cv.dnn.blobFromImage(img, 1.0/255.0,(96,96),(0, 0, 0), True, False)
        open_face.setInput(blob_f)
        vec = open_face.forward()


        vec = cv.normalize(vec, None,-1, 1, cv.NORM_MINMAX)
    elif model_dnn_type =="FACENET":
        
        blob_f = cv.dnn.blobFromImage(img, 1.0/255.0,(160,160),(0, 0, 0), True, False)
        face_net.setInput(blob_f)
        vec = face_net.forward()


        vec = cv.normalize(vec, None,-1, 1, cv.NORM_MINMAX)
    return vec

In [91]:
def TrainModelfull():
    
    image_labels,files,indice_etiqueta_lista = ExtractDatasetData()
    
    indexes_ok=[]
    array_keypoints=[]
    for i,f in enumerate(files):

        img, detections = FindFaceinImage(path_image=f)
        if detections is not None:
            indexes_ok.append(i)
            boxes = ExtractFaceRectangle(img,detections,0.5)
            for b in boxes:
                #DrawImageFaceRectangle(img,b[0],b[1],)
                (startX, startY, endX, endY) = b[0]
                points = ExtractFacePoints(img[startY:endY,startX:endX])
                if points is not None:
                    array_keypoints.append(points)

    #             cv.imshow("Output", img[startY:endY,startX:endX])
    #             cv.waitKey(0)
    #             cv.destroyAllWindows()
        #break


    [indice_etiqueta_lista[i] for i in indexes_ok]

    x_train = np.array([arr.flatten() for arr in array_keypoints])
    y_train = np.array(indice_etiqueta_lista)
    y_train = y_train.reshape(y_train.shape[0],1).ravel()

    TrainSVC(x_train,y_train)
    model_name = "../modeloSVM/etiquetas.pkl"
    with open(model_name, 'wb') as file:  
        pickle.dump(image_labels, file)

    print("Train finished")
    

In [157]:
r = ReconhecimentoFacial("OPENFACE")

In [153]:
r = None

In [142]:
r.TrainModelfull()

TypeError: TrainModelfull() takes 0 positional arguments but 1 was given

In [159]:
img = cv.imread("bolso_teste.jpg")
img2 = r.PredictFace(img)
cv.imshow("Output", img2)
cv.waitKey(0)
cv.destroyAllWindows()

In [123]:
    def PredictFace(self, img):
        img, detections = self.FindFaceinImage(img=img)
        if detections is not None:
            boxes = self.ExtractFaceRectangle(img,detections,0.4)
            #print(boxes)
            for b in boxes:
                (startX, startY, endX, endY) = b[0]
                points = self.ExtractFacePoints(img[startY:endY,startX:endX])
                if points is not None:
                    print("passou")
                    if not self.svc.probability:
                        id_ = self.svc.predict(points)
                        df = self.svc.decision_function(points)
                        result = np.max(softmax(df))
                        etiqueta = self.image_labels[id_[0]]
                        
                    else:
                        id_ = self.svc.classes_[np.argmax(self.svc.predict_proba(points))]
                        result = np.max(self.svc.predict_proba(points))
                        etiqueta = self.image_labels[id_]
                    
                    img = self.DrawImageFaceRectangle(img,b[0],result,etiqueta)
            return img


In [133]:
del(img)
import gc
gc.collect()

NameError: name 'img' is not defined